In [69]:
import numpy as np
from pyvis.network import Network
import networkx as nx
from networkx import drawing as dw
import matplotlib.pyplot as plt
%matplotlib qt5

In [70]:
LAYOUT = lambda g: dw.layout.spring_layout(g, seed=42, iterations=60)
# LAYOUT = lambda g: dw.layout.spectral_layout(g)
# LAYOUT = lambda g: dw.layout.planar_layout(g)

In [56]:
def init():
    global C, N, S, G, count
    count = 0
    C = np.zeros((10,10), dtype='int')
    # N = np.random.randint(2, size=(10,10), )
    N = np.array([
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]])
    S = N.copy()
    G = nx.DiGraph(S)

def init2(size = 10):
    global C, N, S, G, count
    count = 0
    C = np.zeros((size,size), dtype='int')
    N = np.random.randint(2, size=(size,size), ) & np.random.randint(2, size=(size,size), )
    S = N.copy()
    G = nx.DiGraph(S)

def init15example():
    global C, N, S, G, count
    count = 0
    C = np.zeros((15,15), dtype='int')
    N = np.array([[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0]])
    S = N.copy()
    G = nx.DiGraph(S)

def reset_N_and_C():
    global C, N, S, G, count
    count = 0
    C[:] = 0
    N = S.copy()
    S = N.copy()
    G = nx.DiGraph(S)

def initempty(size = 10):
    global C, N, S, G, count
    count = 0
    C = np.zeros((size,size), dtype='int')
    N = np.zeros((size,size), dtype='int')
    S = N.copy()
    G = nx.DiGraph(S)


def init_from_nx(graph):
    global C, N, S, G, count
    count = 0
    N = nx.to_numpy_array(graph, dtype='int')
    C = np.zeros_like(N)
    S = N.copy()
    G = nx.DiGraph(S)
    # G = graph

init()

In [57]:
fig = plt.figure(1, figsize=(10,10))
plt.tight_layout()
def drawnext(s=None, explore=False, filter_c=None, base_only=False):
    global G, count
    
    plt.clf()
    layout = LAYOUT(G)
    # nx.draw_circular(G, with_labels=True, font_weight='bold', edge_color='blue', width=10, alpha=.3, )
    nx.draw_networkx_edges(G, layout, edge_color='blue', width=5, alpha=.5, node_size=400, arrowsize=35, )
    nx.draw_networkx_nodes(G, layout, node_color='w', edgecolors='k', node_size=400)
    nx.draw_networkx_labels(G, layout)
    if base_only: return
    
    Gn = nx.DiGraph(N)
    Gc = nx.DiGraph(C)
    nx.draw_networkx_edges(Gc, layout, edge_color='k', alpha=.9, width=3, arrowsize=15,connectionstyle='arc3,rad=0.15',
                           edgelist=Gc.edges(filter_c))
    nx.draw_networkx_edges(Gn, layout, edge_color='red', width=3,
                           alpha=.6, arrowstyle='-[', min_target_margin=15,
                           connectionstyle='arc3,rad=0.05')
    if s is not None:
        select_color = 'red' if explore else 'blue'
        nx.draw_networkx_nodes(G, layout, node_color=select_color, edgecolors='k', node_size=700, nodelist=[s])
    count+=1
        
    plt.title(str(count))
    plt.pause(.01)
drawnext()

In [58]:
def explore(s, root):
    drawnext(s, explore=True)
    for t in N[s].nonzero()[0]:
        if not C[root,t]: # needs to be checked as many times as N has ones.
            C[root, t] = 1 # after exploring, this node known by the root.
            # N[s,t] = 0 # pop this front (covered in N update below)
            explore(t, root)
            C[s] |= C[t] # pull target's cover into current node
            C[s,t] = 1 # and add the target itself to the cover
            N[s] = ~C[s] & (N[s] | N[t])
            drawnext(s)
    overlap = (C[s] & N[s])
    if s != root:
        assert not overlap.any(), f"{s=} overlap: {np.transpose(overlap.nonzero())}"

### Init

In [68]:
LAYOUT = lambda g: dw.layout.spring_layout(g, seed=42, iterations=40)
# LAYOUT = lambda g: dw.layout.spectral_layout(g)
# LAYOUT = lambda g: dw.layout.planar_layout(g)

# graph = nx.path_graph(50, create_using=nx.DiGraph)
# graph = nx.to_directed(nx.watts_strogatz_graph(20,3, .4))
V = 60
# graph = nx.random_k_out_graph(V, 3, 3)
# graph = nx.binomial_tree(5)
graph = nx.soft_random_geometric_graph(V, .2)
init_from_nx(graph)
drawnext(base_only=True)


### Solve:

In [22]:
start = 34
drawnext(s=start)
explore(start, start)
drawnext()
overlap = (C & N)
if overlap.any():
    print("Overlap at:")
    print(np.transpose(overlap.nonzero()))
plt.pause(1)
drawnext(filter_c=start)    

In [51]:
def solve_kleene():
    global G
    for root in range(G.number_of_nodes()):
        explore(root, root)

# Test Complete Solve Time

In [52]:
# V = 50
# graph = nx.random_k_out_graph(V, 3, 3)
# # graph = nx.binomial_tree(5)
# init_from_nx(graph)
drawnext()
%prun solve_kleene()
drawnext(filter_c=0)

KeyboardInterrupt: 

In [159]:
def shell_example(p=.01):
    start = 19
    graph = nx.to_directed(nx.watts_strogatz_graph(20,5, p, seed=10))
    init_from_nx(graph)
    LAYOUT = lambda g: dw.layout.spring_layout(g, seed=42, iterations=40)
    drawnext(s=start)
    explore(start, start)
    drawnext()
    overlap = (C & N)
    if overlap.any():
        print("Overlap at:")
        print(np.transpose(overlap.nonzero()))
    plt.pause(1)
    drawnext(filter_c=start)
shell_example(.1)

In [139]:
drawnext(filter_c=11)


In [ ]:
nt = Network('1000px', '1000px', notebook=False, directed=True, )
# nt.show_buttons(filter_=[])
nt.set_options("""var options = {
  "edges": {
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.35
      }
    },
    "smooth": false
  },
  "physics": {
    "minVelocity": 0.75
  }
}""")
nt.from_nx(G)
nt.show('a.html')